In [9]:
from pathlib import Path
import sys  
import os
import pandas as pd 
import scipy.sparse as sps
from datetime import datetime

import numpy as np
from operator import itemgetter
import optuna as op

In [10]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM

from scipy.sparse import hstack, vstack
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [13]:
URM_train = sps.load_npz("URM_train_ens.npz")
URM_val = sps.load_npz("URM_val_ens.npz")
URM_test = sps.load_npz("URM_test_ens.npz")

URM_train_val = sps.load_npz("URM_train_val_ens.npz")

print(f"Train size: {URM_train.getnnz()},  Val size: {URM_val.getnnz()}, Test size: {URM_test.getnnz()}")

evaluator_validation = EvaluatorHoldout(URM_val, cutoff_list=[10])

Train size: 1274929,  Val size: 224987, Test size: 264691
EvaluatorHoldout: Ignoring 1008 ( 2.8%) Users that have less than 1 test interactions


In [14]:
from libs.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender

In [18]:
def objective_function_PureSVDItem(optuna_trial):
    recommender_instance = PureSVDItemRecommender(URM_train)
    recommender_instance.fit(
        topK = optuna_trial.suggest_int('topK', 2, 100, log=True),
        num_factors = optuna_trial.suggest_int('num_factors', 10, 1000, log=True),
    )
    
    eval_res, _ = evaluator_validation.evaluateRecommender(recommender_instance)  
    
    trial_results = {
        "RECALL": eval_res["RECALL"][10],
        "MAP": eval_res["MAP"][10]  # Save MAP for reference
    }
    
    # Save the MAP value to the optuna trial user attributes for later use
    optuna_trial.set_user_attr("MAP", trial_results["MAP"])
    
    # Return RECALL for optimization
    return trial_results["MAP"]

In [19]:
class SaveResults(object):
    def __init__(self, file_path=None):
        self.file_path = file_path
        self.results = []
    
    def __call__(self, optuna_study, optuna_trial):
        trial_result = optuna_trial.params.copy()
        trial_result["iteration"] = optuna_trial.number
        trial_result["RECALL"] = optuna_trial.value
        
        # Retrieve the MAP metric from user attributes
        trial_result["MAP"] = optuna_trial.user_attrs.get("MAP")
        
        self.results.append(trial_result)
        
        # Convert to DataFrame and save to CSV
        results_df = pd.DataFrame(self.results)
        results_df.to_csv(self.file_path, index=False)


In [20]:
optuna_study_PureSVDItem = op.create_study(direction="maximize")
        
save_results_PureSVDItem = SaveResults("result_experiments/PureSVDItem/results_optuna.csv")
        
optuna_study_PureSVDItem.optimize(objective_function_PureSVDItem,
                      callbacks=[save_results_PureSVDItem],
                      n_trials = 400)

[I 2025-01-06 21:56:55,073] A new study created in memory with name: no-name-3d7f2a8a-c5ab-4312-83cb-981e5cb4d074


PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!
EvaluatorHoldout: Processed 34728 (100.0%) in 12.21 sec. Users per second: 2845


[I 2025-01-06 21:57:53,737] Trial 0 finished with value: 0.017079408272178003 and parameters: {'topK': 26, 'num_factors': 459}. Best is trial 0 with value: 0.017079408272178003.


PureSVDItemRecommender: Computing SVD decomposition...


[W 2025-01-06 21:58:31,733] Trial 1 failed with parameters: {'topK': 2, 'num_factors': 690} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/miniconda3/envs/RecSys/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/var/folders/tr/d1rx7zt15lzgkp2yc1nwbg100000gn/T/ipykernel_68757/2049785235.py", line 3, in objective_function_PureSVDItem
    recommender_instance.fit(
  File "/Users/safuan/Python/RecSysCompetition2024Polimi/libs/Recommenders/MatrixFactorization/PureSVDRecommender.py", line 135, in fit
    W_sparse = compute_W_sparse_from_item_latent_factors(VT.T, topK=topK)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/safuan/Python/RecSysCompetition2024Polimi/libs/Recommenders/MatrixFactorization/PureSVDRecommender.py", line 75, in compute_W_sparse_from_item_latent_factors
    this_block_we

KeyboardInterrupt: 